In [ ]:
import sqlite3
import re
conn = sqlite3.connect("users.db")
cursor = conn.cursor()
def check_password(pw):
    errors = []
    if len(pw) < 8:
        errors.append("超過8個字元")
    if not re.search(r"[A-Z]", pw) or not re.search(r"[a-z]", pw):
        errors.append("包含英文大小寫")
    if not re.search(r"[!@#$%^&*(),.?\":{}|<>]", pw):
        errors.append("包含特殊字元")
    for i in range(len(pw)-2):
        if ord(pw[i])+1 == ord(pw[i+1]) and ord(pw[i])+2 == ord(pw[i+2]):
            errors.append("不可包含連號")
            break
    if errors:
        return "密碼必須" + "且".join(errors) + "，重新輸入"
    return True
while True:
    a=input("輸入後進入模式 (a) sign up / (b) sign in")[0]
    if(a=='a'):
        name=input("請輸入姓名")
        email=input("請輸入電子郵件")
        last_10=email[-10:]
        while(last_10!="@gmail.com"):
            email=input("Email 格式不符重新輸入")
            last_10=email[-10:]
            if(last_10=="@gmail.com"):
                break
        password = input("請輸入密碼: ")
        pw_check = check_password(password)
        while pw_check != True:
            password = input(pw_check)
            pw_check = check_password(password)
        confirm=input("儲存  "+name+"  |  "+email+"  |  "+password+"  |  Y / N ? (Y更新/儲存資料，N返回進入模式畫面)").upper()
        if(confirm=='Y'):
            cursor.execute("SELECT * FROM user_data WHERE email=?", (email,))
            existing=cursor.fetchone()
            if existing:
                update_confirm=input("Email 已存在，是否更新此 Email 資訊？Y/N ").upper()
                if(update_confirm=='Y'):
                    cursor.execute("UPDATE user_data SET name=?, password=? WHERE email=?", (name, password, email))
                    print("資料已更新")
                else:
                    print("未更新資料")
            else:
                cursor.execute("INSERT INTO user_data (name, email, password) VALUES (?, ?, ?)", (name, email, password))
                print("註冊成功")

            conn.commit()
            break 

        else:
            continue
    elif a == 'b':
        name = input("請輸入姓名")
        email = input("請輸入電子郵件")
        cursor.execute("SELECT password FROM user_data WHERE name=? AND email=?", (name, email))
        result = cursor.fetchone()
        if result is None:
            print("名字或 Email 錯誤")
            continue
        else:
            db_password = result[0]
            password = input("請輸入密碼: ")
            while password != db_password:
                confirm = input("密碼錯誤，忘記密碼 Y / N ? ").upper()
                if confirm == 'Y':
                    print("請重新註冊")
                    break
                else:
                    password = input("請重新輸入密碼: ")
            else:
                print("登入成功")
                break
    else:
        continue
conn.commit()
conn.close()
